<a href="https://colab.research.google.com/github/Center-for-Atmospheric-Research-ATMOS/deposition-calculator/blob/main/deposition_calculator_prod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Here the way to estimate the coverage of the sample in the electrostatic precipitator is demostrated.

# Method

As an input, the method requires deposition time $t$, sample linear size $D_{plate}$ and voltage $V_{plate}$ on it during precipitation, and $dN/dlog(D_p)$ particle size distribution which we further translate to $dN$.

The coverage can be estimated with:

$$coverage = \frac{\sum_{i=1}^m c(d_{pi})\cdot \pi d_{pi}^2}{A_s}$$

where $d_{pi}$ represents the midpoint diameter of the bin in a statistical particle size distribution, $c(d_{pi})$ is the concentration of particles on the sample substrate in the corresponding statistical bin, $m$ - number of bins.
In this equation we assume spherical particles which do not deposit on each other until the whole substrate is fully covered. Spherical shape allow to estimate the area covered by one particle as the area of the cross-section.

The concentration $c(d_{pi})$ for each particles size was calculated using the approach proposed by [Preger et al, 2020](https://doi.org/10.1080/02786826.2020.1716939):

$$c(d_{pi}) = dN(d_{pi}) \cdot Z_{pi} \cdot E_{plate} \cdot t $$

where $dN(d_{pi})$ number of particles in a bin with midpoint diameter $d_{pi}$, $Z_{pi}$ - particle electrical mobility, $E_p$ - electric field strength at the sample substrate.

The electrical mobility of particles, [Welker 2012](https://www.sciencedirect.com/science/article/abs/pii/B9781437778830000018):

$$Z_{pi} = \frac{n\cdot e \cdot C_c(d_{pi})}{3 \pi \eta d_{pi}}$$

where $n$ is a number of elementary charges $e$ on the particle, $C_c(d_{pi})$ - Cunningham slip correction, $\eta$ is the viscosity of the medium.
We assume one charge particles, so $ne$ is equal to one elementary charge.

The Cunningham slip correction $C_c(d_{pi})$ which is a correction to the friction for particles between the continuum and free molecular regime [Cunningham, 1910](https://doi.org/10.1098/rspa.1910.0024), [Welker 2012](https://www.sciencedirect.com/science/article/abs/pii/B9781437778830000018):

$$C_c = 1 + \frac{2 \lambda}{d_{pi}}(1.142 + 0.558e^{-\frac{0.999d_{pi}}{2 \lambda}})$$

where $\lambda$ is the mean free path in medium.

Finally, the average electric field strength at the precipitator plate can be estimated from its definition as:

$$E_{plate} = \frac{V_{plate}}{D_{plate}}$$

where $V_{plate}$ is votage at the substrate during the deposition process, $D_{plate}$ is a substrate diameter or linear size.

In [ ]:
#@markdown # Run to import required Python packages
# interaction
from google.colab import files

# calculations
import numpy as np
import pandas as pd
import random
from scipy.integrate import simps
from scipy.optimize import curve_fit
import scipy.stats as stats

# work with text
import re

# visualisation
import plotly.express as px

In [ ]:
#@markdown # Upload file
#@markdown Run the cell to upload particle size distribution from AIM in *.TXT
#@markdown format.

datafile = files.upload();

Saving 20240204_MgCl2_4.TXT to 20240204_MgCl2_4.TXT


# Check particle size distribution

In [ ]:
#@markdown Function to read AIM txt file
def read_file(filename, column=1):
    '''Retrieves info from the AIM txt file.
    in the returned dataframe diameter is [nm], counts are [particles/cm^3]

    Args:
    filename: last file uploaded to google drive
    column: chosen column with counts, 1 by default

    Returns:
    df - pd.dataframe containing particle size distribution
    '''
    # read file
    filename = list(datafile.keys())[0]
    with open(filename, 'r', errors='ignore') as f:
        text = f.read()

    # get bins from AIM
    match = re.search(
        r'(?<=Diameter Midpoint \(nm\)\n)(.|\n)*?(?=\nScan Time \(s\))',
        text)
    match = match.group().strip().replace('\t\t', '\t')

    # convert bins to float, add bins to df
    df = pd.DataFrame(
        [i.split(',') for i in match.split('\n')])
    df = df.dropna()
    display(df)
    df = df.astype('float')

    print(f"Data contains {len(df.columns) - 1} columns")
    print(f"Here the column {column} is shown\n")
    df = df[[0, column]]

    # rename columns
    df = df.rename(columns={0: "Diameter", column: "Counts"})
    return df

In [ ]:
#@markdown Function describing lognormal probability density.
#@markdown It is required if curve fitting is planned to be applied.
def lognormal_pdf(x, mu, sigma, scale):
    """Calculates the lognormal probability density function.

    Args:
    x: A NumPy array of particle diameters.
    mu: The mean of the lognormal distribution.
    sigma: The standard deviation of the lognormal distribution.
    scale: The factor that affecting the amplitude the distribution

    Returns:
    A NumPy array of the lognormal probability density function.
    """
    return  scale / (x * sigma * np.sqrt(2 * np.pi)) * np.exp(-(np.log10(x) - mu)**2 / (2 * sigma**2))

In [ ]:
#@markdown Charge correction (if needed)

def charge_correction(Dp):
    '''Applies the formula from Wiedensohler, 1988
    coefficients are for +1 charged particles

    Args:
    Dp: array of particle diameters

    Returns:
    dN_corr: corrected dN
    '''
    a0 = -2.3484
    a1 = 0.6044
    a2 = 0.48
    a3 = 0.0013
    a4 = -0.1544
    a5 = 0.0320

    return 10**(a0 + a1 * np.log10(Dp) + a2 * np.log10(Dp)**2 + a3 * np.log10(Dp)**3\
     + a4 * np.log10(Dp)**4 + a5 * np.log10(Dp)**5)

The cell below read the data from the AIM file.
However, the file formats changes depending on the software version, which leads to incorrect file parsing.
In this case this part will return an error

```
column = 1
filename = list(datafile.keys())[0]
size_distribution = read_file(filename, column)
```

There are several ways to deal with that.

First, is simply to hardcode the input, creating your own dataframe:

```
size_distribution = pd.DataFrame()
size_distribution["Diameter"] = [1, 2, 3, 4, 5]
size_distribution["Counts"] = [213, 1434, 123, 3454, 3413]
```

Second way, is upload an Excel file containing Diameter and Counts columns, then read it:

```
size_distribution = pd.read_excel(file_name)
```

The third way is to update the function reading AIM files so that it would parse files properly.


In [ ]:
# read file
column = 1
filename = list(datafile.keys())[0]
size_distribution = read_file(filename, column)

# get diameter and counts to make code clearer
Dp = np.array(size_distribution["Diameter"])
dNdlogDp = np.array(size_distribution["Counts"])

# Calculate the mean logarithmic width of each size bin
dlogDp = np.mean(np.log10(Dp[1:]) - np.log10(Dp[:-1]))

# Multiply the dN/dlog(Dp) values by the logarithmic width to get dN
dN_raw = dNdlogDp * dlogDp

# charge correction, uncomment if needed
#dN_raw = dN_raw / charge_correction(Dp)

size_distribution["dN_raw"] = dN_raw
size_distribution["dN"] = dN_raw

# plot figure
fig = px.scatter(x=Dp, y=dN_raw, log_x=True)

fig.update_layout(
    xaxis_title="Dp [nm]",
    yaxis_title="dN [particles/cm^3]",
    title="Particle size distribution",
    width=800,
    height=400
)

fig.show()

,0,1
0,17.5,925255.361
2,18.1,1002053.666
4,18.8,1019206.841
6,19.5,1234745.479
8,20.2,1422912.706
...,...,...
182,461.4,11824.338
184,478.3,9178.757
186,495.8,5933.613
188,514.0,7183.846


Data contains 1 columns
Here the column 1 is shown



If the expected one peak with lognormal distribution can't be seen, it is possible to fit the data using the cell below.
For example, the distribution may look incorrect because of the concentration higher than the limit of CPC.
To perform the fitting presumably incorrect data points will be removed and the rest will be fitted with one peak lognormal probability density function.


In [ ]:
# Dp-range to exclude. Data taken from the figure above.
# 2 ranges case
exclude_indices = (
    ((Dp >= 50) & (Dp <= 50) | (Dp >= 181) & (Dp <= 573))
    )
# 1 range case
#exclude_indices = (Dp >= 90) & (Dp <= 259)

# initital guess for fitting
# change if the fit does not converge
initial_guess = [0, 1, max(dN_raw)]

# fit a lognormal distribution
params, covariance = curve_fit(
    lognormal_pdf,
    Dp[~exclude_indices],
    dN_raw[~exclude_indices],
    p0=initial_guess)

# extract best fitting parameters
mu, sigma, scale = params

# get final dN in [particles/cm^3], add raw dN
size_distribution["dN_raw"] = dN_raw
size_distribution["dN"] = lognormal_pdf(Dp, mu, sigma, scale)

# plot result
fig = px.scatter(size_distribution,
                 x="Diameter",
                 y=["dN_raw", "dN"],
                 log_x=True)

fig.update_layout(
    xaxis_title="Dp [nm]",
    yaxis_title="dN [particles/cm^3]",
    title="Particle size distribution",
    width=800,
    height=400)

fig.show()

Input experimental metadata in the cell below, then run the cell.

In [ ]:
# INPUT METADATA AND RUN THE CELL

MFP = 6.73e-8 # [m] mean free path

eta = 1.83245e-5 # [Pa∙s] the viscosity of the medium

DEPOSITION_TIME = 20*60 # [s], exposure time

SAMPLE_DIAMETER = 1e-2 # [m], diameter of the subsrtate

SAMPLE_AREA = .64*1e-4 # [m^2], sample area

Vplate = 1e4 # [V], voltage at the precipitator plate

# [V/m] electric field strength at the precipitator plate
Ep = Vplate / SAMPLE_DIAMETER

In [ ]:
#@markdown # Calculate coverage
#@markdown Run the cell to get coverage
# diameter is converted from [nm] to [m]
# dN is converted from [#/cm^3] to [#/m^3]

# add Cunningham slip correction
size_distribution['Cc'] = size_distribution['Diameter'].apply(
    lambda x: 1 + MFP/(x*1e-9) * (2.514 + 0.8*np.exp(-0.55*(x*1e-9)/MFP)))

# add electrical mobility, assume one charge
size_distribution['Zp'] = size_distribution.apply(
    lambda df: 1.6e-19 * 1 * df.Cc / (3 * np.pi * eta * df.Diameter * 1e-9),
    axis=1)

# add drift velocity
size_distribution['Drift_velocity'] = size_distribution.apply(
    lambda df: df.Zp * Ep,
    axis=1)

# add concentration at the spot after deposition for each particle size
size_distribution['Cspot'] = size_distribution.apply(
    lambda df: df.dN * 1e6 * df.Drift_velocity * DEPOSITION_TIME,
    axis=1)

# add total covered area for each particle size
size_distribution['Covered_area'] = size_distribution.apply(
    lambda df: df.Cspot * SAMPLE_AREA * np.pi * (df.Diameter * 1e-9)**2,
    axis=1)

# calculate total coverage
coverage = size_distribution.Covered_area.sum() / SAMPLE_AREA
print(f'Total coverage is {coverage*100:.2f}%')

# check
size_distribution

Total coverage is 97.38%


,Diameter,Counts,dN_raw,dN,Cc,Zp,Drift_velocity,Cspot,Covered_area
0,17.5,925255.361,14448.847786,14448.847786,13.334713,7.059311e-07,0.193406,3.353389e+12,2.064857e-07
2,18.1,1002053.666,15648.135102,15648.135102,12.913216,6.609560e-07,0.181084,3.400349e+12,2.239807e-07
4,18.8,1019206.841,15916.000197,15916.000197,12.455549,6.137928e-07,0.168162,3.211767e+12,2.282389e-07
6,19.5,1234745.479,19281.865561,19281.865561,12.030818,5.715803e-07,0.156597,3.623387e+12,2.770217e-07
8,20.2,1422912.706,22220.297194,22220.297194,11.635597,5.336470e-07,0.146205,3.898453e+12,3.198342e-07
...,...,...,...,...,...,...,...,...,...
182,461.4,11824.338,184.649630,184.649630,1.369381,2.749562e-09,0.000753,1.669169e+08,7.144721e-09
184,478.3,9178.757,143.336065,143.336065,1.355995,2.626482e-09,0.000720,1.237708e+08,5.693101e-09
186,495.8,5933.613,92.659686,92.659686,1.343139,2.509755e-09,0.000688,7.645581e+07,3.778795e-09
188,514.0,7183.846,112.183406,112.183406,1.330738,2.398535e-09,0.000657,8.846327e+07,4.699148e-09


# References


*   Instruction Manual: Model 3010 Condensation Particle Counter (2002). TSI Incorporated, St. Paul, MN, USA.
*   Operation and Service Manual: Series 3080 Electrostatic Classifiers (2006). TSI Incorporated, St. Paul, MN, USA.
*   E. O. Knutson and K. T. Whitby (1975). Aerosol classification by electric mobility: Apparatus, theory, and applications. J. Aerosol Sci. 6: 443-451.
*   Preger, C., Overgaard, N.C., Messing, M.E. and Magnusson, M.H. (2020). Predicting the deposition spot radius and the nanoparticle concentration distribution in an electrostatic precipitator. Aerosol Science and Technology, 54(6), pp.718-728, http://doi.org/10.1080/02786826.2020.1716939
*   R.W. Welker, R. Kohli, K.L. Mittal (Eds.), Developments in Surface Contamination and Cleaning, William Andrew Publishing, Oxford (2012). pp. 1-80, http://doi.org/10.1016/B978-1-4377-7883-0.00001-8
*   Cunningham Ebenezer (1910). On the velocity of steady fall of spherical particles through fluid medium. Proc. R. Soc. Lond. A83, pp 357–365,
http://doi.org/10.1098/rspa.1910.0024

